In [1]:
import os
from tqdm import tqdm

import numpy as np
import pandas as pd

from PIL import Image

import torch
import torch.nn.functional as F

# 有 GPU 就用 GPU，没有就用 CPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('device', device)


device cuda:0


图像预处理


In [3]:
from torchvision import transforms
# 测试集图像预处理-RCTN：缩放、裁剪、转 Tensor、归一化
test_transform = transforms.Compose([transforms.Resize(256),
                                     transforms.CenterCrop(224),
                                     transforms.ToTensor(),
                                     transforms.Normalize(
                                         mean=[0.485, 0.456, 0.406],
                                         std=[0.229, 0.224, 0.225])
                                    ])


载入测试集（和训练代码教程相同）


In [4]:
# 数据集文件夹路径
dataset_dir = 'animals_split'
test_path = os.path.join(dataset_dir, 'val')
from torchvision import datasets
# 载入测试集
test_dataset = datasets.ImageFolder(test_path, test_transform)
print('测试集图像数量', len(test_dataset))
print('类别个数', len(test_dataset.classes))
print('各类别名称', test_dataset.classes)
# 载入类别名称 和 ID索引号 的映射字典
idx_to_labels = np.load('idx_to_labels.npy', allow_pickle=True).item()
# 获得类别名称
classes = list(idx_to_labels.values())
print(classes)


测试集图像数量 1068
类别个数 89
各类别名称 ['乌鸦', '乌龟', '仓鼠', '企鹅', '兔子', '刺猬', '啄木鸟', '土狼', '大猩猩', '大象', '奶牛', '山羊', '斑马', '松鼠', '毛毛虫', '水母', '水獭', '河马', '浣熊', '海星', '海豚', '海豹', '海马', '火烈鸟', '火鸡', '熊', '熊猫', '牛', '牡蛎', '犀牛', '犀鸟', '狐狸', '狗', '狮子', '狼', '猩猩', '猪', '猫', '猫头鹰', '獾', '瓢虫', '甲虫', '章鱼', '羊', '羚羊', '老虎', '老鼠', '考拉', '苍蝇', '蚊子', '蚱蜢', '蛇', '蛾', '蜂鸟', '蜜蜂', '蜥蜴', '蜻蜓', '蝙蝠', '蝴蝶', '蟑螂', '蟹', '袋熊', '袋鼠', '豪猪', '豹', '负鼠', '野牛', '野猪', '金鱼', '霍加皮', '马', '驯鹿', '驴', '鬣狗', '鱿鱼', '鲨鱼', '鲸鱼', '鸭', '鸽子', '鹅', '鹈鹕', '鹦鹉', '鹬', '鹰', '鹿', '麻雀', '黑猩猩', '鼠', '龙虾']
['乌鸦', '乌龟', '仓鼠', '企鹅', '兔子', '刺猬', '啄木鸟', '土狼', '大猩猩', '大象', '奶牛', '山羊', '斑马', '松鼠', '毛毛虫', '水母', '水獭', '河马', '浣熊', '海星', '海豚', '海豹', '海马', '火烈鸟', '火鸡', '熊', '熊猫', '牛', '牡蛎', '犀牛', '犀鸟', '狐狸', '狗', '狮子', '狼', '猩猩', '猪', '猫', '猫头鹰', '獾', '瓢虫', '甲虫', '章鱼', '羊', '羚羊', '老虎', '老鼠', '考拉', '苍蝇', '蚊子', '蚱蜢', '蛇', '蛾', '蜂鸟', '蜜蜂', '蜥蜴', '蜻蜓', '蝙蝠', '蝴蝶', '蟑螂', '蟹', '袋熊', '袋鼠', '豪猪', '豹', '负鼠', '野牛', '野猪', '金鱼', '霍加皮', '马', '驯鹿', '驴', '鬣狗', '鱿鱼', '鲨鱼', '鲸

导入训练好的模型


In [5]:
model = torch.load('checkpoints/best-0.895.pth')
model = model.eval().to(device)


表格A-测试集图像路径及标注



In [6]:
test_dataset.imgs[:10]


[('animals_split\\val\\乌鸦\\197c1012ba.jpg', 0),
 ('animals_split\\val\\乌鸦\\1ecc081899.jpg', 0),
 ('animals_split\\val\\乌鸦\\2caed0387c.jpg', 0),
 ('animals_split\\val\\乌鸦\\376fae5ad1.jpg', 0),
 ('animals_split\\val\\乌鸦\\495ee7079c.jpg', 0),
 ('animals_split\\val\\乌鸦\\5bd195cf29.jpg', 0),
 ('animals_split\\val\\乌鸦\\6d60806c01.jpg', 0),
 ('animals_split\\val\\乌鸦\\6dab0b74ff.jpg', 0),
 ('animals_split\\val\\乌鸦\\77fefab966.jpg', 0),
 ('animals_split\\val\\乌鸦\\8b94b5d529.jpg', 0)]

In [7]:
img_paths = [each[0] for each in test_dataset.imgs]


In [8]:
df = pd.DataFrame()
df['图像路径'] = img_paths
df['标注类别ID'] = test_dataset.targets
df['标注类别名称'] = [idx_to_labels[ID] for ID in test_dataset.targets]


In [9]:
df


,图像路径,标注类别ID,标注类别名称
0,animals_split\val\乌鸦\197c1012ba.jpg,0,乌鸦
1,animals_split\val\乌鸦\1ecc081899.jpg,0,乌鸦
2,animals_split\val\乌鸦\2caed0387c.jpg,0,乌鸦
3,animals_split\val\乌鸦\376fae5ad1.jpg,0,乌鸦
4,animals_split\val\乌鸦\495ee7079c.jpg,0,乌鸦
...,...,...,...
1063,animals_split\val\龙虾\dc43d4b46c.jpg,88,龙虾
1064,animals_split\val\龙虾\de356673e0.jpg,88,龙虾
1065,animals_split\val\龙虾\ea19158455.jpg,88,龙虾
1066,animals_split\val\龙虾\eaaa72b4e1.jpg,88,龙虾


表格B-测试集每张图像的图像分类预测结果，以及各类别置信度


In [10]:
# 记录 top-n 预测结果
n = 3


In [11]:
df_pred = pd.DataFrame()
for idx, row in tqdm(df.iterrows()):
    img_path = row['图像路径']
    img_pil = Image.open(img_path).convert('RGB')
    input_img = test_transform(img_pil).unsqueeze(0).to(device) # 预处理
    pred_logits = model(input_img) # 执行前向预测，得到所有类别的 logit 预测分数
    pred_softmax = F.softmax(pred_logits, dim=1) # 对 logit 分数做 softmax 运算

    pred_dict = {}

    top_n = torch.topk(pred_softmax, n) # 取置信度最大的 n 个结果
    pred_ids = top_n[1].cpu().detach().numpy().squeeze() # 解析出类别

    # top-n 预测结果
    for i in range(1, n+1):
        pred_dict['top-{}-预测ID'.format(i)] = pred_ids[i-1]
        pred_dict['top-{}-预测名称'.format(i)] = idx_to_labels[pred_ids[i-1]]
    pred_dict['top-n预测正确'] = row['标注类别ID'] in pred_ids
    # 每个类别的预测置信度
    for idx, each in enumerate(classes):
        pred_dict['{}-预测置信度'.format(each)] = pred_softmax[0][idx].cpu().detach().numpy()

    df_pred = df_pred.append(pred_dict, ignore_index=True)


1068it [00:58, 18.38it/s]


In [12]:
df_pred


,top-1-预测ID,top-1-预测名称,top-2-预测ID,top-2-预测名称,top-3-预测ID,top-3-预测名称,top-n预测正确,乌鸦-预测置信度,乌龟-预测置信度,仓鼠-预测置信度,...,鹅-预测置信度,鹈鹕-预测置信度,鹦鹉-预测置信度,鹬-预测置信度,鹰-预测置信度,鹿-预测置信度,麻雀-预测置信度,黑猩猩-预测置信度,鼠-预测置信度,龙虾-预测置信度
0,0.0,乌鸦,30.0,犀鸟,83.0,鹰,1.0,0.62409204,5.7089397e-05,3.960545e-06,...,0.0240693,0.004872751,0.0011937084,0.00043772126,0.029289432,0.0002210004,0.0055506774,0.0005904294,0.0004436836,4.6173645e-06
1,77.0,鸭,78.0,鸽子,0.0,乌鸦,1.0,0.09595812,5.485807e-07,1.5676331e-06,...,0.04427672,0.0022551445,0.00035208167,0.00015695929,0.0028310164,0.00018244985,0.042417698,2.522623e-05,5.8904996e-05,1.5373966e-07
2,0.0,乌鸦,78.0,鸽子,6.0,啄木鸟,1.0,0.9935409,3.293732e-08,1.8917921e-08,...,2.8461836e-05,3.236827e-05,0.00020189308,1.9516895e-06,6.8449954e-05,2.714902e-07,0.0005240315,2.6782375e-06,5.9254674e-08,7.59576e-09
3,0.0,乌鸦,78.0,鸽子,6.0,啄木鸟,1.0,0.9935422,2.438844e-07,4.7850147e-08,...,1.8611076e-05,4.295955e-05,1.0557129e-05,1.06321595e-05,0.0010665988,1.1799222e-06,0.0006836885,7.6174856e-06,5.213173e-07,2.7790225e-07
4,83.0,鹰,0.0,乌鸦,30.0,犀鸟,1.0,0.18843515,5.147259e-06,2.1537948e-08,...,0.0006057041,0.0063642454,0.00018673188,4.524062e-06,0.7659334,8.273503e-07,8.859905e-06,7.9337355e-05,1.0769833e-06,2.6680604e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1063,88.0,龙虾,60.0,蟹,59.0,蟑螂,1.0,3.453474e-10,3.2764396e-09,1.0872233e-09,...,1.4279539e-09,2.613506e-09,4.4026964e-09,8.515743e-11,2.0430582e-10,4.0533705e-09,4.7579265e-09,6.2406125e-10,3.2188523e-09,0.99897647
1064,88.0,龙虾,60.0,蟹,56.0,蜻蜓,1.0,6.79151e-08,2.936218e-07,3.7791272e-09,...,2.1524198e-07,5.4483263e-07,7.4433686e-08,6.269045e-08,3.0604514e-07,5.96325e-07,4.3452446e-06,1.13680825e-08,5.5919497e-08,0.9976295
1065,88.0,龙虾,60.0,蟹,74.0,鱿鱼,1.0,2.8715414e-09,4.4415795e-09,3.108521e-09,...,9.239284e-09,1.6083183e-08,5.57919e-08,9.524478e-10,1.1292259e-08,2.2446065e-09,2.7057894e-08,1.0397476e-09,2.168128e-09,0.97359496
1066,88.0,龙虾,60.0,蟹,74.0,鱿鱼,1.0,3.2633007e-10,8.568386e-09,3.8827083e-10,...,2.772367e-09,1.6431795e-07,3.0059123e-08,2.5100195e-09,1.5430163e-09,1.1378797e-08,2.0243078e-08,2.2770925e-09,1.2519901e-08,0.9981528


拼接AB两张表格


In [13]:
df = pd.concat([df, df_pred], axis=1)


In [14]:
df


,图像路径,标注类别ID,标注类别名称,top-1-预测ID,top-1-预测名称,top-2-预测ID,top-2-预测名称,top-3-预测ID,top-3-预测名称,top-n预测正确,...,鹅-预测置信度,鹈鹕-预测置信度,鹦鹉-预测置信度,鹬-预测置信度,鹰-预测置信度,鹿-预测置信度,麻雀-预测置信度,黑猩猩-预测置信度,鼠-预测置信度,龙虾-预测置信度
0,animals_split\val\乌鸦\197c1012ba.jpg,0,乌鸦,0.0,乌鸦,30.0,犀鸟,83.0,鹰,1.0,...,0.0240693,0.004872751,0.0011937084,0.00043772126,0.029289432,0.0002210004,0.0055506774,0.0005904294,0.0004436836,4.6173645e-06
1,animals_split\val\乌鸦\1ecc081899.jpg,0,乌鸦,77.0,鸭,78.0,鸽子,0.0,乌鸦,1.0,...,0.04427672,0.0022551445,0.00035208167,0.00015695929,0.0028310164,0.00018244985,0.042417698,2.522623e-05,5.8904996e-05,1.5373966e-07
2,animals_split\val\乌鸦\2caed0387c.jpg,0,乌鸦,0.0,乌鸦,78.0,鸽子,6.0,啄木鸟,1.0,...,2.8461836e-05,3.236827e-05,0.00020189308,1.9516895e-06,6.8449954e-05,2.714902e-07,0.0005240315,2.6782375e-06,5.9254674e-08,7.59576e-09
3,animals_split\val\乌鸦\376fae5ad1.jpg,0,乌鸦,0.0,乌鸦,78.0,鸽子,6.0,啄木鸟,1.0,...,1.8611076e-05,4.295955e-05,1.0557129e-05,1.06321595e-05,0.0010665988,1.1799222e-06,0.0006836885,7.6174856e-06,5.213173e-07,2.7790225e-07
4,animals_split\val\乌鸦\495ee7079c.jpg,0,乌鸦,83.0,鹰,0.0,乌鸦,30.0,犀鸟,1.0,...,0.0006057041,0.0063642454,0.00018673188,4.524062e-06,0.7659334,8.273503e-07,8.859905e-06,7.9337355e-05,1.0769833e-06,2.6680604e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1063,animals_split\val\龙虾\dc43d4b46c.jpg,88,龙虾,88.0,龙虾,60.0,蟹,59.0,蟑螂,1.0,...,1.4279539e-09,2.613506e-09,4.4026964e-09,8.515743e-11,2.0430582e-10,4.0533705e-09,4.7579265e-09,6.2406125e-10,3.2188523e-09,0.99897647
1064,animals_split\val\龙虾\de356673e0.jpg,88,龙虾,88.0,龙虾,60.0,蟹,56.0,蜻蜓,1.0,...,2.1524198e-07,5.4483263e-07,7.4433686e-08,6.269045e-08,3.0604514e-07,5.96325e-07,4.3452446e-06,1.13680825e-08,5.5919497e-08,0.9976295
1065,animals_split\val\龙虾\ea19158455.jpg,88,龙虾,88.0,龙虾,60.0,蟹,74.0,鱿鱼,1.0,...,9.239284e-09,1.6083183e-08,5.57919e-08,9.524478e-10,1.1292259e-08,2.2446065e-09,2.7057894e-08,1.0397476e-09,2.168128e-09,0.97359496
1066,animals_split\val\龙虾\eaaa72b4e1.jpg,88,龙虾,88.0,龙虾,60.0,蟹,74.0,鱿鱼,1.0,...,2.772367e-09,1.6431795e-07,3.0059123e-08,2.5100195e-09,1.5430163e-09,1.1378797e-08,2.0243078e-08,2.2770925e-09,1.2519901e-08,0.9981528


In [15]:
df.to_csv('测试集预测结果.csv', index=False)
